In [1]:
import seaborn as sns

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import warnings


warnings.filterwarnings("ignore")

# Modeling boosted tree regression
from sklearn.model_selection import train_test_split


pd.set_option('display.max_columns', None)


In [2]:
import xgboost as xgb

In [3]:
# IMPORT DATA

data = pd.read_csv('../Austin/ames_house_price_no_dummies.csv')
data.head()

# The dataset has many categorical columns. 
#Normally, we'd encode them with ordinal or one-hot encoding, but XGBoost has the ability to internally deal with categoricals.


,pid,gr_liv_area,sale_price,ms_sub_class,ms_zoning,lot_frontage,lot_area,alley,lot_shape,land_contour,lot_config,land_slope,neighborhood,bldg_type,house_style,overall_qual,overall_cond,roof_style,exterior1st,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmt_fin_type1,bsmt_fin_sf1,bsmt_fin_type2,bsmt_fin_sf2,bsmt_unf_sf,total_bsmt_sf,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abv_gr,kitchen_abv_gr,kitchen_qual,tot_rms_abv_grd,functional,fireplaces,fireplace_qu,garage_type,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3_ssn_porch,screen_porch,pool_area,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,sale_condition,bc_sale_price,near_rr,near_main_rd,near_pos,house_age_at_sale,construction_age_at_sale
0,909176150,856,126000,30,RL,80.0,7890,NaN,4,Lvl,Corner,3,SWISU,1Fam,1Story,6,6,Gable,Wd Sdng,None,0.0,3,3,CBlock,3,3,1,3,238.0,1,0.0,618.0,856.0,3,1,4,856,0,0,1.0,0.0,1,0,2,1,3,4,7,1,4,Detchd,1,2.0,399.0,3,3,3,0,0,0,0,166,0,0,NaN,0,3,2010,WD,Normal,10.299527,0,0,0,71,60
1,905476230,1049,139500,120,RL,42.0,4235,NaN,4,Lvl,Inside,3,Edwards,TwnhsE,1Story,5,5,Gable,HdBoard,BrkFace,149.0,4,3,CBlock,4,3,2,6,552.0,5,393.0,104.0,1049.0,3,1,4,1049,0,0,1.0,0.0,2,0,2,1,4,5,7,0,0,Attchd,3,1.0,266.0,3,3,3,0,105,0,0,0,0,0,NaN,0,2,2009,WD,Normal,10.377252,0,0,0,25,25
2,911128020,1001,124900,30,C (all),60.0,6060,NaN,4,Lvl,Inside,3,IDOTRR,1Fam,1Story,5,9,Hip,MetalSd,None,0.0,4,3,BrkTil,3,3,1,5,737.0,1,0.0,100.0,837.0,5,1,4,1001,0,0,0.0,0.0,1,0,2,1,4,5,7,0,0,Detchd,1,1.0,216.0,3,1,1,154,0,42,86,0,0,0,NaN,0,11,2007,WD,Normal,10.292822,0,0,0,77,0
3,535377150,1039,114000,70,RL,80.0,8146,NaN,4,Lvl,Corner,3,OldTown,1Fam,2Story,4,8,Gable,MetalSd,None,0.0,4,4,BrkTil,2,3,1,1,0.0,1,0.0,405.0,405.0,4,1,4,717,322,0,0.0,0.0,1,0,2,1,3,6,7,0,0,Detchd,1,1.0,281.0,3,3,1,0,0,168,0,111,0,0,NaN,0,5,2009,WD,Normal,10.222923,0,0,0,109,6
4,534177230,1665,227000,60,RL,70.0,8400,NaN,4,Lvl,Inside,3,NWAmes,1Fam,2Story,8,6,Gable,VinylSd,None,0.0,4,3,PConc,4,3,1,6,643.0,1,0.0,167.0,810.0,5,1,4,810,855,0,1.0,0.0,2,1,3,1,4,6,7,0,0,Attchd,3,2.0,528.0,3,3,3,0,45,0,0,0,0,0,NaN,0,11,2009,WD,Normal,10.746564,0,0,0,8,8


In [4]:
# Define the target and independent features
X = data.drop(['pid', 'sale_price', 'bc_sale_price'], axis = 1)
X['ms_sub_class'] = X['ms_sub_class'].astype('category')
X['lot_shape'] = X['lot_shape'].astype('category')
X['land_slope'] = X['land_slope'].astype('category')
X['overall_qual'] = X['overall_qual'].astype('category')
X['overall_cond'] = X['overall_cond'].astype('category')
X['exter_qual'] = X['exter_qual'].astype('category')
X['exter_cond'] = X['exter_cond'].astype('category')
X['bsmt_cond'] = X['bsmt_cond'].astype('category')
X['bsmt_qual'] = X['bsmt_qual'].astype('category')
X['bsmt_exposure'] = X['bsmt_exposure'].astype('category')
X['bsmt_fin_type1'] = X['bsmt_fin_type1'].astype('category')
X['bsmt_fin_type2'] = X['bsmt_fin_type2'].astype('category')
X['heating_qc'] = X['heating_qc'].astype('category')
X['electrical'] = X['electrical'].astype('category')
X['kitchen_qual'] = X['kitchen_qual'].astype('category')
X['functional'] = X['functional'].astype('category')
X['fireplace_qu'] = X['fireplace_qu'].astype('category')
X['garage_finish'] = X['garage_finish'].astype('category')
X['garage_qual'] = X['garage_qual'].astype('category')
X['garage_cond'] = X['garage_cond'].astype('category')
X['paved_drive'] = X['paved_drive'].astype('category')


y = data['sale_price']
# Cast categorical variables as pandas categories so that XGBoost handles them as categoricals
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')

In [5]:
cats

['ms_sub_class',
 'ms_zoning',
 'alley',
 'lot_shape',
 'land_contour',
 'lot_config',
 'land_slope',
 'neighborhood',
 'bldg_type',
 'house_style',
 'overall_qual',
 'overall_cond',
 'roof_style',
 'exterior1st',
 'mas_vnr_type',
 'exter_qual',
 'exter_cond',
 'foundation',
 'bsmt_qual',
 'bsmt_cond',
 'bsmt_exposure',
 'bsmt_fin_type1',
 'bsmt_fin_type2',
 'heating_qc',
 'electrical',
 'kitchen_qual',
 'functional',
 'fireplace_qu',
 'garage_type',
 'garage_finish',
 'garage_qual',
 'garage_cond',
 'paved_drive',
 'misc_feature',
 'sale_type',
 'sale_condition']

In [6]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [12]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": 'hist', 'n_jobs': -1}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
)

## EVALUATE

In [14]:
def evaluate(model, test_features, test_labels):
    
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    error_percent_sale_price = (predictions - test_labels) / test_labels
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Test Error: {:0.4f} $USD'.format(np.mean(errors)))
    print('Test Accuracy = {:0.2f}%'.format(accuracy))
    return (accuracy, error_percent_sale_price)


In [15]:
from sklearn.metrics import mean_squared_error
preds_test = model.predict(dtest_reg)

rmse = mean_squared_error(y_test, preds, squared=False)

#mse = np.mean((y_test - preds) ** 2)
#rmse = np.sqrt(mse)

print(f"RMSE of the base model: $ {rmse:.3f}")


evaluate(model, dtest_reg, y_test)

RMSE of the base model: $ 24748.463
Model Performance
Average Test Error: 15390.5078 $USD
Test Accuracy = 90.90%


(90.89580355919556,
 1299   -0.052716
 205     0.388399
 1907    0.009625
 517    -0.117850
 1301   -0.140628
           ...   
 1286    0.019258
 2271   -0.095639
 303    -0.054711
 221     0.261145
 2285   -0.097649
 Name: sale_price, Length: 645, dtype: float64)